# Uganda Flood Fraction by Sentinel-1 Acquisition (per-region) 

This notebook computes, for each Sentinel-1 acquisition intersecting Uganda (2020–2025), the flooded area and coverage percentage per district (Admin-2). Results are appended to a CSV on disk.

**Warning:** This notebook performs server-side Earth Engine computations and transfers per-image × per-district tables client-side. For the full country × 6 years this may be heavy; consider running per-year or smaller chunks.

## Environment Setup

In [10]:
from pathlib import Path
import ee, geemap
import numpy as np
import geopandas as gpd
import pandas as pd
import datetime
from shapely.geometry import mapping

## Setup Google Earth Engine

In [37]:
ee.Authenticate()

Enter verification code:  4/1Ab32j92_hrFPQMbQK_56yOH8ebGxEsKNhsy7hleCKtClrE4gpOnXwNpvRXM



Successfully saved authorization token.


In [44]:
ee.Initialize(project = "divine-catalyst-330916")

In [ ]:
# Parameters
DISTRICT_SHP = 'uganda_admin2.shp'  # path to Admin-2 shapefile
START = '2020-01-01'
END = '2025-08-31'
BASELINE_START = '2017-01-01'
BASELINE_END = '2019-12-31'
ANOMALY_THRESHOLD = -3.0
PERM_WATER_OCCURRENCE_PCT = 50
S1_COLLECTION_ID = 'COPERNICUS/S1_GRD'
OUTPUT_CSV = '/mnt/data/uganda_flood_by_image_district.csv'  # output on disk
SCALE = 10  # meters for Sentinel-1 reductions


In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping

districts_gdf = gpd.read_file(DISTRICT_SHP)
districts_gdf = districts_gdf.to_crs(epsg=4326)
# compute area in m2 and attach as property
districts_gdf['area_m2'] = districts_gdf.geometry.to_crs(epsg=3857).area
# choose name field
name_field = None
for c in ['NAME_2','district','ADM2_NAME','name','DN']:
    if c in districts_gdf.columns:
        name_field = c
        break
if name_field is None:
    name_field = districts_gdf.columns[0]

# Convert to ee FeatureCollection with area property
import geemap
districts_fc = geemap.geopandas_to_ee(districts_gdf[[name_field,'geometry','area_m2']])
# ensure features have 'district_name' and 'area_m2'
def rename_props(feature):
    return feature.set({'district_name': feature.get(name_field), 'area_m2': feature.get('area_m2')})
districts_fc = districts_fc.map(rename_props)
print('Loaded', districts_gdf.shape[0], 'districts; name field =', name_field)

In [ ]:
import ee
S1 = ee.ImageCollection(S1_COLLECTION_ID)
GSW = ee.Image('JRC/GSW1_3/GlobalSurfaceWater')
print('Collections loaded')

In [ ]:
# Build Sentinel-1 image list intersecting Uganda in timeframe
s1_filtered = (S1
               .filterDate(START, END)
               .filter(ee.Filter.eq('instrumentMode', 'IW'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filterBounds(districts_fc.geometry()))
# get list of ids and times (server-side reduction to lists)
info = s1_filtered.reduceColumns(ee.Reducer.toList(2), ['system:index','system:time_start']).getInfo()
ids = info['list'][0]
times = info['list'][1]
print('Found', len(ids), 'images')

In [ ]:
import datetime, time
from tqdm import tqdm

# If output csv exists, we will append; otherwise write header
import os
if not os.path.exists(OUTPUT_CSV):
    with open(OUTPUT_CSV, 'w') as f:
        header = ['image_id','acq_datetime','district_name','flooded_m2','district_area_m2','flood_fraction','coverage_pct','baseline_exists']
        f.write(','.join(header) + '\n')

# iterate images (client-side loop) and for each process per-district server-side, then fetch results
for iid, t in tqdm(list(zip(ids, times)), total=len(ids)):
    acq_time = datetime.datetime.utcfromtimestamp(int(t)/1000)
    acq_str = acq_time.strftime('%Y-%m-%dT%H:%M:%SZ')
    print('Processing', iid, acq_str)
    img = ee.Image('COPERNICUS/S1_GRD/' + iid).select('VV')
    month = acq_time.month

    # server-side per-feature function
    def per_feature_fn(feature):
        geom = feature.geometry()
        # baseline collection for that district and month
        baseline_col = (S1
                        .filterDate(BASELINE_START, BASELINE_END)
                        .filter(ee.Filter.eq('instrumentMode','IW'))
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VV'))
                        .filterBounds(geom)
                        .filter(ee.Filter.calendarRange(month, month, 'month'))
                        .select('VV'))
        baseline_exists = baseline_col.size().gt(0)
        baseline = baseline_col.median().clip(geom)

        img_clip = img.clip(geom)
        anomaly = img_clip.subtract(baseline)

        perm_water = GSW.select('occurrence').gte(PERM_WATER_OCCURRENCE_PCT)
        flood_mask = anomaly.lte(ANOMALY_THRESHOLD).And(perm_water.Not())

        # flooded area (m2)
        flooded_area = ee.Image.pixelArea().updateMask(flood_mask).reduceRegion(
            ee.Reducer.sum(), geom, SCALE, maxPixels=1e13).get('area')

        # valid (covered) area = pixelArea masked by img_clip.mask()
        valid_area = ee.Image.pixelArea().updateMask(img_clip.mask()).reduceRegion(
            ee.Reducer.sum(), geom, SCALE, maxPixels=1e13).get('area')

        district_area = ee.Number(feature.get('area_m2'))

        # compute coverage percent and fraction, guard against nulls
        flooded_area_num = ee.Number(flooded_area).unmask(0)
        valid_area_num = ee.Number(valid_area).unmask(0)
        coverage_pct = ee.Number(0)
        # if district_area > 0: coverage = valid_area/district_area *100
        coverage_pct = ee.Algorithms.If(district_area.gt(0), valid_area_num.divide(district_area).multiply(100), ee.Number(0))
        flood_fraction = ee.Algorithms.If(district_area.gt(0), flooded_area_num.divide(district_area), ee.Number(0))

        return feature.set({
            'image_id': iid,
            'acq_time': acq_str,
            'flooded_m2': flooded_area_num,
            'district_area_m2': district_area,
            'flood_fraction': flood_fraction,
            'coverage_pct': coverage_pct,
            'baseline_exists': baseline_exists
        })

    # map server-side
    results_fc = districts_fc.map(per_feature_fn)

    # bring results client-side (safe: number of districts ~100-200)
    try:
        feats = results_fc.getInfo()['features']
    except Exception as e:
        print('Error getInfo for image', iid, e)
        continue

    # append to CSV
    with open(OUTPUT_CSV, 'a') as f:
        for feat in feats:
            props = feat['properties']
            line = [
                props.get('image_id',''),
                props.get('acq_time',''),
                str(props.get('district_name','')).replace(',', ';'),
                str(props.get('flooded_m2',0)),
                str(props.get('district_area_m2',0)),
                str(props.get('flood_fraction',0)),
                str(props.get('coverage_pct',0)),
                str(props.get('baseline_exists',False))
            ]
            f.write(','.join(line) + '\n')

    # sleep briefly to be polite to EE servers and avoid throttling
    time.sleep(0.5)

print('Done. Output saved to', OUTPUT_CSV)

## Notes

- This notebook appends results to a CSV on disk (`OUTPUT_CSV`).
- It computes `coverage_pct` as the fraction of district area that had any valid Sentinel-1 pixels in that image (pixelArea of mask / district area).
- The approach uses server-side baseline median computed per district/month on-the-fly.
- For long runs, consider chunking by year and exporting each chunk separately.